**INITIALIZATION:**
- I use these three lines of code on top of my each notebooks because it will help to prevent any problems while reloading the same project. And the third line of code helps to make visualization within the notebook.

In [1]:
#@ INITIALIZATION: 
%reload_ext autoreload
%autoreload 2
%matplotlib inline

**LIBRARIES AND DEPENDENCIES:**
- I have downloaded all the libraries and dependencies required for the project in one particular cell.

In [2]:
#@ IMPORTING NECESSARY LIBRARIES AND DEPENDENCIES: 
from torch.nn import ConvTranspose2d
from torch.nn import BatchNorm2d
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import LeakyReLU
from torch.nn import ReLU
from torch.nn import Tanh
from torch.nn import Sigmoid
from torch import flatten
from torch import nn
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision import transforms
from sklearn.utils import shuffle
import imutils
from imutils import build_montages
from torch.optim import Adam
from torch.nn import BCELoss
import numpy as np
import torch
import cv2
import os

**DCGANs**
- Deep Convolutional Generative Adversarial Networks (DCGANs) was introduced by Radford et al. in their 2016 paper - *Unsupervised Representation Learning with Deep Convolutional Generative Adversarial Networks*. DCGANs at that time showed us how to effectively use convolutional techniques with GANs without supervision to create images that are quite similar to those in our dataset. 

In [3]:
#@ INITIALIZING GENERATOR MODULE:
class Generator(nn.Module):                                                     # Defining Generator Module. 
    def __init__(self, inputDim=100, outputChannels=1):                         # Initializing Constructor Function. 
        super(Generator, self).__init__()                                       # Initializing Super Constructor. 
        self.ct1 = ConvTranspose2d(in_channels=inputDim, out_channels=128, 
                                   kernel_size=4, stride=2, padding=0, 
                                   bias=False)                                  # Initializing Transposed Convolution. 
        self.relu1 = ReLU()                                                     # Initializing RELU Activation. 
        self.batchNorm1 = BatchNorm2d(128)                                      # Initializing Batch Normalization. 
        self.ct2 = ConvTranspose2d(in_channels=128, out_channels=64, 
                                   kernel_size=3, stride=2, padding=1,
                                   bias=False)                                  # Adding Transposed Convolution. 
        self.relu2 = ReLU()                                                     # Adding RELU Activation Function. 
        self.batchNorm2 = BatchNorm2d(64)                                       # Adding Batch Normalization Layer.
        self.ct3 = ConvTranspose2d(in_channels=64, out_channels=32, 
                                   kernel_size=4, stride=2, padding=1, 
                                   bias=False)                                  # Adding Transposed Convolution. 
        self.relu3 = ReLU()                                                     # Adding RELU Activation Function. 
        self.batchNorm3 = BatchNorm2d(32)                                       # Adding Batch Normalization Layer. 
        self.ct4 = ConvTranspose2d(in_channels=32,out_channels=outputChannels, 
                                   kernel_size=4, stride=2, padding=1, 
                                   bias=False)                                  # Adding Transposed Convolution. 
        self.tanh = Tanh()                                                      # Adding RELU Activation Function. 

    def forward(self, x):                                                       # Defining Forward Method. 
        x = self.ct1(x)                                                         # Transposed Convolution. 
        x = self.relu1(x)                                                       # RELU Activation Function.
        x = self.batchNorm1(x)                                                  # Batch Normalization Layer. 
        x = self.ct2(x)                                                         # Transposed Convolution. 
        x = self.relu2(x)                                                       # RELU Activation Function.
        x = self.batchNorm2(x)                                                  # Batch Normalization Layer. 
        x = self.ct3(x)                                                         # Transposed Convolution. 
        x = self.relu3(x)                                                       # RELU Activation Function.
        x = self.batchNorm3(x)                                                  # Batch Normalization Layer. 
        x = self.ct4(x)                                                         # Transposed Convolution. 
        output = self.tanh(x)                                                   # Tanh Activation Function. 
        return output

**DISCRIMINATOR:**
- Generator module is going to model random noise into an image. Discriminator takes the image and outputs a single value. 

In [4]:
#@ INITIALIZING DISCRIMINATOR MODULE: 
class Discriminator(nn.Module):                                         # Defining Discriminator Module. 
    def __init__(self, depth, alpha=0.2):                               # Initializing Constructor Function. 
        super(Discriminator, self).__init__()                           # Initializing Super Constructor. 
        self.conv1 = Conv2d(in_channels=depth, out_channels=32, 
                            kernel_size=4, stride=2, padding=1)         # Initializing Convolutional Layer. 
        self.leakyRelu1 = LeakyReLU(alpha, inplace=True)                # Initializing Leaky RELU. 
        self.conv2 = Conv2d(in_channels=32, out_channels=64, 
                            kernel_size=4, stride=2, padding=1)         # Adding Convolutional Layer. 
        self.leakyRelu2 = LeakyReLU(alpha, inplace=True)                # Adding Leaky RELU. 
        self.fc1 = Linear(in_features=3136, out_features=512)           # Adding Linear FC Layer. 
        self.leakyRelu3 = LeakyReLU(alpha, inplace=True)                # Adding Leaky RELU. 
        self.fc2 = Linear(in_features=512, out_features=1)              # Adding Linear Output Layer. 
        self.sigmoid = Sigmoid()                                        # Adding Sigmoid Layer. 
    
    def forward(self, x):                                               # Defining Forward Method. 
        x = self.conv1(x)                                               # Adding Convolutional Layer. 
        x = self.leakyRelu1(x)                                          # Leaky RELU Activation. 
        x = self.conv2(x)                                               # Adding Convolutional Layer. 
        x = self.leakyRelu2(x)                                          # Leaky RELU Activation. 
        x = flatten(x, 1)                                               # Adding Flatten Layer. 
        x = self.fc1(x)                                                 # Adding Linear Layer. 
        x = self.leakyRelu3(x)                                          # Leaky RELU Activation. 
        x = self.fc2(x)                                                 # Linear Output Layer. 
        output = self.sigmoid(x)                                        # Sigmoid Activation Function. 
        return output

**TRAINING DCGANs**

In [5]:
#@ CUSTOM WEIGHTS INITIALIZATION FUNCTION: 
def weights_init(model):                                # Defining Weight Initialization Function. 
    classname = model.__class__.__name__                # Initializing Model Name.
    if classname.find("Conv") != -1:
        nn.init.normal_(model.weight.data, 0.0, 0.02)   # Initializing Weights for Convolutional Layer.
    elif classname.find("BatchNorm") != -1:
        nn.init.normal_(model.weight.data, 1.0, 0.02)   # Initializing Weights for Batch Normalization Layer. 
        nn.init.constant_(model.bias.data, 0)           # Initializing Bias for Batch Normalization Layer. 

In [7]:
#@ INITIALIZING PARAMETERS: 
NUM_EPOCHS = 20                                                             # Initializing Epoch Size. 
BATCH_SIZE = 128                                                            # Initializing Batch Size. 
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")     # Initializing GPU. 

#@ INITIALIZING DATA TRANSFORMATIONS: 
dataTransforms = transforms.Compose([transforms.ToTensor(),                 # Converting into Tensors. 
                                     transforms.Normalize((0.5), (0.5))])   # Normalizing the Data. 

#@ INITIALIZING DATASET: 
trainData = MNIST(root="data", train=True, download=True, 
                  transform=dataTransforms)                                 # Initializing Training Data. 
testData = MNIST(root="data", train=False, download=True,
                 transform=dataTransforms)                                  # Initializing Test Data.
data = torch.utils.data.ConcatDataset((trainData, testData))                # Stacking the Dataset. 

#@ INITIALIZING DATALOADERS:
dataloader = DataLoader(data, shuffle=True, batch_size=BATCH_SIZE)          # Initializing DataLoader. 

In [10]:
#@ TRAINING DCGANs:
stepsPerEpoch = len(dataloader.dataset) // BATCH_SIZE                       # Initialization. 
gen = Generator(inputDim=100, outputChannels=1)                             # Initializing Generator. 
gen.apply(weights_init)                                                     # Initializing Weights. 
gen.to(DEVICE)                                                              # Loading into GPU. 
disc = Discriminator(depth=1)                                               # Initializing Discriminator. 
disc.apply(weights_init)                                                    # Initializing Weights. 
disc.to(DEVICE)                                                             # Loading into GPU. 

#@ INITIALIZING OPTIMIZERS: 
genOpt = Adam(gen.parameters(), lr=0.0002, betas=(0.5, 0.999), 
              weight_decay=0.0002 / NUM_EPOCHS)                             # Generator Optimizer. 
discOpt = Adam(disc.parameters(), lr=0.0002, betas=(0.5, 0.999), 
              weight_decay=0.0002 / NUM_EPOCHS)                             # Discriminator Optimizer. 
criterion = BCELoss()                                                       # Initializing Binary Cross Entropy Loss Function. 

#@ TRAINING DCGANs:
benchmarkNoise = torch.randn(256, 100, 1, 1, device=DEVICE)                 # Generating Noise.
realLabel, fakeLabel = 1, 0                                                 # Initialization. 
for epoch in range(NUM_EPOCHS):
    print("Starting epoch {} of {}...".format(epoch + 1, NUM_EPOCHS))       # Inspecting Epochs. 
    epochLossG, epochLossD = 0, 0                                           # Initializing Loss for Generator & Discriminator. 
    for x in dataloader:
        disc.zero_grad()                                                    # Zeroing Discriminator Gradients. 
        images = x[0]                                                       # Getting Images. 
        images = images.to(DEVICE)                                          # Loading into GPU.
        bs = images.size(0)                                                 # Initializing Batch Size. 
        labels = torch.full((bs,), realLabel, dtype=torch.float, 
                            device=DEVICE)                                  # Initializing Labels. 
        output = disc(images).view(-1)                                      # Forward Pass and Reshaping. 
        errorReal = criterion(output, labels)                               # Calculating Loss. 
        errorReal.backward()                                                # Calculating Gradients. 
        noise = torch.randn(bs, 100, 1, 1, device=DEVICE)                   # Initialzing Noise for Generator. 
        fake = gen(noise)                                                   # Generating Fake Image. 
        labels.fill_(fakeLabel)
        output = disc(fake.detach()).view(-1)                               # Forward Pass and Reshaping. 
        errorFake = criterion(output, labels)                               # Calculating Loss. 
        errorFake.backward()                                                # Computing Gradients. 
        errorD = errorReal + errorFake                                      # Computing Error for Discriminator. 
        discOpt.step()                                                      # Updating. 
        gen.zero_grad()                                                     # Zeroing Gradients. 
        labels.fill_(realLabel)
        output = disc(fake).view(-1)                                        # Forward Pass and Reshaping. 
        errorG = criterion(output, labels)                                  # Computing Loss. 
        errorG.backward()                                                   # Computing Gradients. 
        genOpt.step()                                                       # Updating. 
        epochLossD += errorD
        epochLossG += errorG
    print("Generator Loss: {:.4f}, Discriminator Loss: {:.4f}".format(
        epochLossG / stepsPerEpoch, epochLossD / stepsPerEpoch))
    if (epoch + 1) % 2 == 0:
        gen.eval()                                                          # Generator in Evaluation Phase. 
        images = gen(benchmarkNoise)                                        # Generating Predictions. 
        images = images.detach().cpu().numpy().transpose((0, 2, 3, 1))
        images = ((images * 127.5) + 127.5).astype("uint8")
        images = np.repeat(images, 3, axis=-1)
        vis = build_montages(images, (28, 28), (16, 16))[0]                 # Building Montage. 
        p = os.path.join("./data/", "epoch_{}.png".format(
            str(epoch + 1).zfill(4)))                                       # Building Patches. 
        cv2.imwrite(p, vis)
        gen.train()                                                         # Generator in Training Mode. 

Starting epoch 1 of 20...
Generator Loss: 4.2994, Discriminator Loss: 0.4664
Starting epoch 2 of 20...
Generator Loss: 1.1703, Discriminator Loss: 1.0551
Starting epoch 3 of 20...
Generator Loss: 0.9579, Discriminator Loss: 1.1877
Starting epoch 4 of 20...
Generator Loss: 0.8817, Discriminator Loss: 1.2405
Starting epoch 5 of 20...
Generator Loss: 0.8739, Discriminator Loss: 1.2506
Starting epoch 6 of 20...
Generator Loss: 0.8890, Discriminator Loss: 1.2483
Starting epoch 7 of 20...
Generator Loss: 0.8814, Discriminator Loss: 1.2529
Starting epoch 8 of 20...
Generator Loss: 0.8886, Discriminator Loss: 1.2565
Starting epoch 9 of 20...
Generator Loss: 0.8848, Discriminator Loss: 1.2547
Starting epoch 10 of 20...
Generator Loss: 0.8898, Discriminator Loss: 1.2540
Starting epoch 11 of 20...
Generator Loss: 0.9032, Discriminator Loss: 1.2445
Starting epoch 12 of 20...
Generator Loss: 0.9131, Discriminator Loss: 1.2403
Starting epoch 13 of 20...
Generator Loss: 0.9258, Discriminator Loss: 1.